# Qiskit Runtime on IBM Cloud

Qiskit Runtime is now part of the IBM Quantum Services on IBM Cloud. To use this service, you'll need to create an IBM Cloud account and a quantum service instance. [This guide](https://cloud.ibm.com/docs/account?topic=account-account-getting-started) contains step-by-step instructions on setting up the account and [this page](https://cloud.ibm.com/docs/quantum-computing?topic=quantum-computing-quickstart) explains how to create a service instance, including directions to find your IBM Cloud API key and Cloud Resource Name (CRN), which you will need later in this tutorial. 



This tutorial assumes that you know how to use Qiskit, including using it to create circuits. If you are not familiar with Qiskit, the [Qiskit Textbook](https://qiskit.org/textbook/preface.html) is a great resource to learn about both Qiskit and quantum computation in general. 

After following the guide above you should have:

 - IBM Cloud API key.
 - IBM Cloud CRN.

You will need this info later in this tutorial to authenticate in the service.

# qiskit-ibm-runtime package

First, you need to install Qiskit if you haven't it already. Once you have an IBM Cloud account and service instance set up, you can use `qiskit-ibm-runtime` to access Qiskit Runtime on IBM Cloud. `qiskit-ibm-runtime` provides the interface to interact with Qiskit Runtime. You can, for example, use it to query and execute runtime programs.

### Before you begin

Make sure you have:

- [An IBM Cloud account](https://cloud.ibm.com/docs/account?topic=account-account-getting-started)
- [QiSkit runtime service instance](https://cloud.ibm.com/docs/quantum-computing?topic=quantum-computing-quickstart)

## Installation

You can install `qiskit` and the `qiskit-ibm-runtime` package using pip:

```
pip install -U qiskit
pip install -U qiskit-ibm-runtime
```

In [1]:
!pip install -U qiskit
!pip install -U qiskit-ibm-runtime

     |████████████████████████████████| 6.7 MB 35.2 MB/s eta 0:00:01
     |████████████████████████████████| 18.0 MB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 240 kB 71.4 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 7.9 MB/s  eta 0:00:01
     |████████████████████████████████| 111 kB 61.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 67.8 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 12.9 MB/s eta 0:00:01
     |████████████████████████████████| 930 kB 71.1 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 17.6 MB/s eta 0:00:01
     |████████████████████████████████| 37.5 MB 26.8 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 79.8 MB/s eta 0:00:01
  Created wheel for qiskit: filename=qiskit-0.37.1-py3-none-any.whl size=12067 sha256=e28800843250700554694e27180f1fff4018cd9572747d0c08d20b51993c12cf
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/fb/40/c5/91f30b4b9c9db7e4b8e099445d70e4

## Account initialization

Before you can start using Qiskit Runtime, you need to initialize your account by calling `QiskitRuntimeService` with your IBM Cloud API key and the CRN or service name of your service instance.

You can also choose to save your credentials on disk (in the `$HOME/.qiskit/qiskit-ibm.json` file). By doing so, you only need to use `QiskitRuntimeService()` in the future to initialize your account.

For more information about account management, such as how to delete or view an account, see or import this notebook [04_account_management.ipynb](https://github.com/Qiskit/qiskit-ibm-runtime/blob/main/docs/tutorials/04_account_management.ipynb). 

<div class="alert alert-block alert-info">
<b>Note:</b> Account credentials are saved in plain text, so only do so if you are using a trusted device. 
</div>

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    channel="ibm_cloud",
    token="",
    instance="",
    overwrite=True
)

service = QiskitRuntimeService()

The `<IBM Cloud API key>` in the example above is your IBM Cloud API key and looks something like

```
kYgdggnD-qx5k2u0AAFUKv3ZPW_avg0eQ9sK75CCW7hw
```

The `<IBM Cloud CRN>` is the Cloud Resource Name and looks something like

```
crn:v1:bluemix:public:quantum-computing:us-east:a/b947c1c5a9378d64aed96696e4d76e8e:a3a7f181-35aa-42c8-94d6-7c8ed6e1a94b::
```

The `<IBM Cloud service name>` is user-provided and defaults to something like
```
Quantum Services-9p
```
If you choose to set `instance` to the service name, the initialization time of the `QiskitRuntimeService` is slightly higher because the required `CRN` value is internally resolved via IBM Cloud APIs.

## Listing programs <a name='listing_program'>

There are three methods that can be used to find metadata of available programs:

- `pprint_programs()`: pretty prints summary metadata of available programs
- `programs()`: returns a list of `RuntimeProgram` instances
- `program()`: returns a single `RuntimeProgram` instance

The metadata of a runtime program includes its ID, name, description, maximum execution time, backend requirements, input parameters, and return values. Maximum execution time is the maximum amount of time, in seconds, a program can run before being forcibly terminated.

To print the summary metadata of the programs (by default first 20 programs are displayed):

In [3]:
service.pprint_programs(limit=10)

hello-world:
  Name: hello-world
  Description: A sample runtime program.
sampler:
  Name: sampler
  Description: Generates quasi-probabilities by sampling quantum circuits.
estimator:
  Name: estimator
  Description: Calculates expectation values of quantum operators.


You can use the `limit` and `skip` parameters in `pprint_programs()` and `programs()` to page through all programs. 

You can pass `detailed=True` parameter to `pprint_programs()` to view all the metadata for the programs. 

The program metadata, once fetched, is cached for performance reasons. But you can pass the `refresh=True` parameter  to get the latest data from the server. 

To print the metadata of the program `sampler`:

In [4]:
program = service.program("sampler")
print(program)

sampler:
  Name: sampler
  Description: Generates quasi-probabilities by sampling quantum circuits.
  Creation date: 2021-10-26T14:41:57Z
  Update date: 2022-07-19T17:21:25.407871Z
  Max execution time: 10000
  Backend requirements:
    none
  Input parameters:
    Properties:
        - circuit_indices:
            Description: Indices of the circuits to evaluate.
            Type: array
            Required: True
        - circuits:
            Description: A single or list of QuantumCircuits.
            Type: ['array', 'object']
            Required: True
        - parameter_values:
            Description: A list of concrete parameters to be bound for each circuit. If specified, it must have the same length as circuit_indices.
            Type: array
            Required: False
        - parameters:
            Description: Parameters of the quantum circuits.
            Type: array
            Required: False
        - resilience_settings:
            Description: Resilience setti

As you can see from above, the primitive `sampler` calculates the distributions generated by given circuits executed on the target backend. It takes a number of parameters, but `circuits` and `circuit_indices` are the only required parameters. When the program finishes, it returns the quasi-probabilities for each circuit.

## Invoking a runtime program <a name='invoking_program'>

You can use the [QiskitRuntimeService.run()](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.QiskitRuntimeService.html#qiskit_ibm_runtime.QiskitRuntimeService.run) method to invoke a runtime program. This method takes the following parameters:

- `program_id`: ID of the program to run.
- `inputs`: Program input parameters. These input values are passed to the runtime program.
- `options`: Runtime options. These options control the execution environment. Currently the only available option is `backend_name`, which is optional for cloud runtime.
- `result_decoder`: Optional class used to decode the job result.

Below is an example of invoking the `sampler` program.

First we need to construct a circuit as the input to `sampler` using Qiskit.

In [5]:
from qiskit import QuantumCircuit

N = 6
qc = QuantumCircuit(N)

qc.x(range(0, N))
qc.h(range(0, N))

for kk in range(N // 2, 0, -1):
    qc.ch(kk, kk - 1)
for kk in range(N // 2, N - 1):
    qc.ch(kk, kk + 1)
qc.measure_all()
qc.draw(fold=-1)

┌───┐┌───┐          ┌───┐ ░ ┌─┐               
   q_0: ┤ X ├┤ H ├──────────┤ H ├─░─┤M├───────────────
        ├───┤├───┤     ┌───┐└─┬─┘ ░ └╥┘┌─┐            
   q_1: ┤ X ├┤ H ├─────┤ H ├──■───░──╫─┤M├────────────
        ├───┤├───┤┌───┐└─┬─┘      ░  ║ └╥┘┌─┐         
   q_2: ┤ X ├┤ H ├┤ H ├──■────────░──╫──╫─┤M├─────────
        ├───┤├───┤└─┬─┘           ░  ║  ║ └╥┘┌─┐      
   q_3: ┤ X ├┤ H ├──■────■────────░──╫──╫──╫─┤M├──────
        ├───┤├───┤     ┌─┴─┐      ░  ║  ║  ║ └╥┘┌─┐   
   q_4: ┤ X ├┤ H ├─────┤ H ├──■───░──╫──╫──╫──╫─┤M├───
        ├───┤├───┤     └───┘┌─┴─┐ ░  ║  ║  ║  ║ └╥┘┌─┐
   q_5: ┤ X ├┤ H ├──────────┤ H ├─░──╫──╫──╫──╫──╫─┤M├
        └───┘└───┘          └───┘ ░  ║  ║  ║  ║  ║ └╥┘
meas: 6/═════════════════════════════╩══╩══╩══╩══╩══╩═
                                     0  1  2  3  4  5

We now use this circuit as the input to `sampler`:

In [6]:
# Specify the program inputs here.
program_inputs = {
    "circuits": qc,
    "circuit_indices": [0],
}

job = service.run(
    program_id="sampler",
    inputs=program_inputs,
)

# Printing the job ID in case we need to retrieve it later.
print(f"Job ID: {job.job_id}")

# Get the job result - this is blocking and control may not return immediately.
result = job.result()
print(result)
# see which backend the job was executed on
print(job.backend) 

Job ID: cblaudfmub8b8qnd7ht0
{'quasi_dists': [{'100001': 0.015625, '000001': 0.0205078125, '110001': 0.0322265625, '100011': 0.0380859375, '111111': 0.5078125, '110000': 0.0361328125, '110011': 0.0615234375, '110111': 0.125, '000000': 0.013671875, '000111': 0.0478515625, '100111': 0.06640625, '000011': 0.0234375, '100000': 0.01171875}], 'metadata': [{'header_metadata': {}, 'shots': 1024}]}
<IBMBackend('ibmq_qasm_simulator')>


### Runtime job

The `run()` method returns a [RuntimeJob](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.RuntimeJob.html#qiskit_ibm_runtime.RuntimeJob) instance, which represents the asynchronous execution instance of the program. 

Some of the `RuntimeJob` methods:

- `status()`: Return job status.
- `result()`: Wait for the job to finish and return the final result.
- `cancel()`: Cancel the job.
- `wait_for_final_state()`: Wait for the job to finish.
- `logs()`: Return job logs.
- `error_message()`: Returns the reason if the job failed and `None` otherwise.

Some of the `RuntimeJob` attributes:

- `job_id`: Unique identifier of the job.
- `backend`: The backend where the job is run.
- `program_id`: ID of the program the execution is for.


Refer to the [RuntimeJob API documentation](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.RuntimeJob.html#qiskit_ibm_runtime.RuntimeJob) for a full list of methods and usage. 

<div class="alert alert-block alert-info">
<b>Note:</b> To ensure fairness, there is a maximum execution time for each Qiskit Runtime job. Refer to <a href="https://qiskit.org/documentation/partners/qiskit_ibm_runtime/max_time.html#qiskit-runtime-on-ibm-cloud">this documentation</a> on what the time limit is.
</div>

## Selecting a backend

A **backend** is a quantum device or simulator capable of running quantum circuits or pulse schedules.

In the example above, we invoked a runtime program without specifying which backend it should run on. In this case the server automatically picks the one that is the least busy. Alternatively, you can choose a specific backend to run your program. 

To list all the backends you have access to:

In [7]:
service.backends()

[<IBMBackend('ibmq_qasm_simulator')>,
 <IBMBackend('simulator_stabilizer')>,
 <IBMBackend('simulator_mps')>,
 <IBMBackend('simulator_extended_stabilizer')>,
 <IBMBackend('simulator_statevector')>]

The [QiskitRuntimeService.backends()](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.QiskitRuntimeService.html#qiskit_ibm_runtime.QiskitRuntimeService.backends) method also takes filters. For example, to find all real devices that have at least five qubits:

In [8]:
service.backends(simulator=False, min_num_qubits=5)

[]

[QiskitRuntimeService.backends()](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.QiskitRuntimeService.html#qiskit_ibm_runtime.QiskitRuntimeService.backends) returns a list of [IBMBackend](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.IBMBackend.html#qiskit_ibm_runtime.IBMBackend) instances. Each instance represents a particular backend. Attributes and methods of an `IBMBackend` provide more information about the backend, such as its qubit count, error rate, and status.

For more information about backends, such as commonly used attributes, see [03_backends.ipynb](https://github.com/Qiskit/qiskit-ibm-runtime/blob/main/docs/tutorials/03_backends.ipynb).

Once you select a backend to use, you can specify the name of the backend in the `options` parameter:

In [10]:
# Specify the program inputs here.
program_inputs = {
    "circuits": qc,
    "circuit_indices": [0],
}

# Specify the backend name.
options = {"backend_name": "ibmq_qasm_simulator"}

job = service.run(
    program_id="sampler",
    options=options,
    inputs=program_inputs,
)

# Printing the job ID in case we need to retrieve it later.
print(f"Job ID: {job.job_id}")

# Get the job result - this is blocking and control may not return immediately.
result = job.result()
print(result)

Job ID: cblaulmp4otpvgfohtag
{'quasi_dists': [{'000001': 0.009765625, '100000': 0.013671875, '110000': 0.0478515625, '110011': 0.0615234375, '110001': 0.0322265625, '100011': 0.0244140625, '111111': 0.4892578125, '100001': 0.025390625, '000000': 0.0107421875, '000011': 0.037109375, '100111': 0.07421875, '000111': 0.0546875, '110111': 0.119140625}], 'metadata': [{'header_metadata': {}, 'shots': 1024}]}


## Retrieving previously run jobs

You can use the [QiskitRuntimeService.job()](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.QiskitRuntimeService.html#qiskit_ibm_runtime.QiskitRuntimeService.job) method to retrieve a previously executed runtime job. Attributes of this [RuntimeJob](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.RuntimeJob.html#qiskit_ibm_runtime.RuntimeJob) instance can tell you about the execution:

In [11]:
retrieved_job = service.job(job.job_id)
print(
    f"Job {retrieved_job.job_id} is an execution instance of runtime program {retrieved_job.program_id}."
)
print(
    f"This job ran on backend {retrieved_job.backend} and had input parameters {retrieved_job.inputs}"
)

Job cblaulmp4otpvgfohtag is an execution instance of runtime program sampler.
This job ran on backend <IBMBackend('ibmq_qasm_simulator')> and had input parameters {'circuits': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7f3cb03dddc0>, 'circuit_indices': [0]}


Similarly, you can use [QiskitRuntimeService.jobs()](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.QiskitRuntimeService.html#qiskit_ibm_runtime.QiskitRuntimeService.jobs) to get a list of jobs. You can specify a limit on how many jobs to return. The default limit is 10:

In [12]:
retrieved_jobs = service.jobs(limit=1)
for rjob in retrieved_jobs:
    print(rjob.job_id)

cblaulmp4otpvgfohtag


## Deleting a job

You can use the [QiskitRuntimeService.delete_job()](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.QiskitRuntimeService.html#qiskit_ibm_runtime.QiskitRuntimeService.delete_job) method to delete a job. You can only delete your own jobs, and this action cannot be reversed. 

In [13]:
service.delete_job(job.job_id)

# Next steps

Now that you are familiar with the basics of Qiskit Runtime, you can find additional tutorials in the [tutorials](https://github.com/Qiskit/qiskit-ibm-runtime/tree/main/docs/tutorials) directory in the public qiskit-ibm-runtime repo:

- [03_backends.ipynb](https://github.com/Qiskit/qiskit-ibm-runtime/blob/main/docs/tutorials/03_backends.ipynb) describes how to find a target backend for the Qiskit Runtime program you want to invoke. 
- [04_account_management.ipynb](https://github.com/Qiskit/qiskit-ibm-runtime/blob/main/docs/tutorials/04_account_management.ipynb) describes how to save, load, and delete your account credentials on disk.
- [qiskit_runtime_vqe_program.ipynb](https://github.com/Qiskit/qiskit-ibm-runtime/blob/main/docs/tutorials/sample_vqe_program/qiskit_runtime_vqe_program.ipynb) goes into more details on uploading a real-world program (VQE). 

To import them into your account, follow these instructions about [importing notebooks from a URL](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/creating-notebooks.html).

In [14]:
from qiskit.tools.jupyter import *

%qiskit_copyright